In [7]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/zhiyinnn/dsa4262-project.git

Cloning into 'dsa4262-project'...
remote: Enumerating objects: 43551, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 43551 (delta 2), reused 0 (delta 0), pack-reused 43540 (from 1)
Receiving objects: 100% (43551/43551), 601.84 MiB | 48.31 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (18352/18352), done.


In [8]:
%cd /content/dsa4262-project
!git branch -a
!git checkout resize-dataset



/content/dsa4262-project
  main
* resize-dataset
  remotes/origin/GAN
  remotes/origin/HEAD -> origin/main
  remotes/origin/SMOTE
  remotes/origin/dataset
  remotes/origin/diffusion
  remotes/origin/main
  remotes/origin/resize-dataset
Already on 'resize-dataset'
Your branch is up to date with 'origin/resize-dataset'.


In [9]:
import pandas as pd
import matplotlib as mat
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

pd.options.display.max_colwidth = 100

import random
import os

from numpy.random import seed
seed(42)

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

import tensorflow as tf
print(tf.test.is_gpu_available())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import glob


from tensorflow.random import set_seed
set_seed(42)

import warnings
warnings.filterwarnings('ignore')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False
Num GPUs Available:  0
2.19.0


In [18]:
IMG_SIZE = 150
BATCH = 32
SEED = 42
main_path_original = "./resize_xray/chest_xray"
main_path_gan = "./resize_xray/chest_xray_gan"
main_path_diffusion = "./resize_xray/chest_xray_diffusion"

train_path = os.path.join(main_path_original,"train")
val_path=os.path.join(main_path_original,"val")
test_path=os.path.join(main_path_original,"test")
og_train_normal = glob.glob(train_path+"/NORMAL/*.jpeg")
og_train_pneumonia = glob.glob(train_path+"/PNEUMONIA/*.jpeg")
og_train_covid19 = glob.glob(train_path+"/COVID19/*.jpg")
og_train_tb = glob.glob(train_path+"/TURBERCULOSIS/*.png")

test_normal = glob.glob(test_path+"/NORMAL/*.jpeg")
test_pneumonia = glob.glob(test_path+"/PNEUMONIA/*.jpeg")
test_covid19 = glob.glob(test_path+"/COVID19/*.jpg")
test_tb = glob.glob(test_path+"/TURBERCULOSIS/*.png")

val_normal = glob.glob(val_path+"/NORMAL/*.jpeg")
val_pneumonia = glob.glob(val_path+"/PNEUMONIA/*.jpeg")
val_covid19 = glob.glob(val_path+"/COVID19/*.jpg")
val_tb = glob.glob(val_path+"/TURBERCULOSIS/*.png")

gan_train_normal = glob.glob(main_path_gan+"/NORMAL/*.png")
gan_train_covid = glob.glob(main_path_gan+"/COVID19/*.png")
gan_train_tb = glob.glob(main_path_gan+"/TB/*.png")

diffusion_train_normal = glob.glob(main_path_diffusion+"/NORMAL/*.png")
diffusion_train_covid = glob.glob(main_path_diffusion+"/COVID19/*.png")
diffusion_train_tb = glob.glob(main_path_diffusion+"/TB/*.png")

og_train_list = [x for x in og_train_normal]
og_train_list.extend([x for x in og_train_pneumonia] + [x for x in og_train_covid19] + [x for x in og_train_tb])

og_df_train = pd.DataFrame(np.concatenate([['Normal']*len(og_train_normal) , ['Pneumonia']*len(og_train_pneumonia), ['COVID-19']*len(og_train_covid19), ['Tuberculosis']*len(og_train_tb)]), columns = ['class'])
og_df_train['image'] = [x for x in og_train_list]

test_list = [x for x in test_normal]
test_list.extend([x for x in test_pneumonia]+ [x for x in test_covid19] + [x for x in test_tb])

df_test = pd.DataFrame(np.concatenate([['Normal']*len(test_normal) , ['Pneumonia']*len(test_pneumonia), ['COVID-19']*len(test_covid19), ['Tuberculosis']*len(test_tb)]), columns = ['class'])
df_test['image'] = [x for x in test_list]

val_list = [x for x in val_normal]
val_list.extend([x for x in val_pneumonia]+ [x for x in val_covid19] + [x for x in val_tb])

df_val = pd.DataFrame(np.concatenate([['Normal']*len(val_normal) , ['Pneumonia']*len(val_pneumonia), ['COVID-19']*len(val_covid19), ['Tuberculosis']*len(val_tb)]), columns = ['class'])
df_val['image'] = [x for x in val_list]

og_df_train['class'].value_counts()

,count
class,
Pneumonia,3875
Normal,1341
Tuberculosis,650
COVID-19,460


In [11]:
df_val['class'].value_counts()

,count
class,
Tuberculosis,12
COVID-19,10
Pneumonia,8
Normal,8


In [19]:
df_test['class'].value_counts()

,count
class,
Pneumonia,390
Normal,234
COVID-19,106
Tuberculosis,41


In [20]:
#getting df for original + gan

gan_train_list = [x for x in gan_train_normal]
gan_train_list.extend([x for x in gan_train_covid] + [x for x in gan_train_tb])

gan_df_train = pd.DataFrame(np.concatenate([['Normal']*len(gan_train_normal) , ['COVID-19']*len(gan_train_covid), ['Tuberculosis']*len(gan_train_tb)]), columns = ['class'])
gan_df_train['image'] = [x for x in gan_train_list]

gan_df_train = pd.concat([og_df_train, gan_df_train], ignore_index=True)
gan_df_train = gan_df_train.sample(frac=1, random_state=42).reset_index(drop=True)

print(gan_df_train['class'].value_counts())
display(gan_df_train)

class
Tuberculosis    3875
Normal          3875
COVID-19        3875
Pneumonia       3875
Name: count, dtype: int64


,class,image
0,Tuberculosis,./resize_xray/chest_xray_gan/TB/image_2946.png
1,Normal,./resize_xray/chest_xray_gan/NORMAL/image_2185.png
2,COVID-19,./resize_xray/chest_xray_gan/COVID19/image_2485.png
3,Normal,./resize_xray/chest_xray_gan/NORMAL/image_425.png
4,COVID-19,./resize_xray/chest_xray_gan/COVID19/image_2503.png
...,...,...
15495,Pneumonia,./resize_xray/chest_xray/train/PNEUMONIA/person672_bacteria_2565.jpeg
15496,Tuberculosis,./resize_xray/chest_xray_gan/TB/image_737.png
15497,COVID-19,./resize_xray/chest_xray/train/COVID19/COVID19(124).jpg
15498,Normal,./resize_xray/chest_xray/train/NORMAL/NORMAL2-IM-0919-0001.jpeg


In [24]:
from sklearn.model_selection import train_test_split

# Stratified 85/15 split by class
gan_train_df, gan_val_df = train_test_split(
    gan_df_train,
    test_size=0.15,
    stratify=gan_df_train['class'],  # preserve class distribution
    random_state=42
)
print("Training set class counts:")
print(gan_train_df['class'].value_counts())
print("\nValidation set class counts:")
print(gan_val_df['class'].value_counts())




Training set class counts:
class
Normal          3294
Tuberculosis    3294
COVID-19        3294
Pneumonia       3293
Name: count, dtype: int64

Validation set class counts:
class
Pneumonia       582
Normal          581
Tuberculosis    581
COVID-19        581
Name: count, dtype: int64


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
#create df for original + diffusion

diffusion_train_list = [x for x in diffusion_train_normal]
diffusion_train_list.extend([x for x in diffusion_train_covid] + [x for x in diffusion_train_tb])

diffusion_df_train = pd.DataFrame(np.concatenate([['Normal']*len(diffusion_train_normal) , ['COVID-19']*len(diffusion_train_covid), ['Tuberculosis']*len(diffusion_train_tb)]), columns = ['class'])
diffusion_df_train['image'] = [x for x in diffusion_train_list]

diffusion_df_train = pd.concat([og_df_train, diffusion_df_train], ignore_index=True)
diffusion_df_train = gan_df_train.sample(frac=1, random_state=42).reset_index(drop=True)

print(diffusion_df_train['class'].value_counts())
display(diffusion_df_train)

class
Pneumonia       3875
COVID-19        3875
Normal          3875
Tuberculosis    3875
Name: count, dtype: int64


,class,image
0,Pneumonia,./resize_xray/chest_xray/train/PNEUMONIA/person568_bacteria_2359.jpeg
1,COVID-19,./resize_xray/chest_xray/train/COVID19/COVID19(395).jpg
2,COVID-19,./resize_xray/chest_xray_gan/COVID19/image_2517.png
3,Normal,./resize_xray/chest_xray/train/NORMAL/IM-0319-0001.jpeg
4,Normal,./resize_xray/chest_xray/train/NORMAL/IM-0683-0001.jpeg
...,...,...
15495,Tuberculosis,./resize_xray/chest_xray_gan/TB/image_539.png
15496,Pneumonia,./resize_xray/chest_xray/train/PNEUMONIA/person723_virus_1342.jpeg
15497,COVID-19,./resize_xray/chest_xray_gan/COVID19/image_2427.png
15498,Tuberculosis,./resize_xray/chest_xray_gan/TB/image_155.png


In [45]:
print(diffusion_df_train['class'].unique())


['Pneumonia' 'COVID-19' 'Normal' 'Tuberculosis']


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255.,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1/255.)

In [26]:
ds_train = train_datagen.flow_from_dataframe(
    dataframe=gan_train_df,
    x_col='image',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH,
    shuffle=True,
    seed=SEED
)

ds_val = val_datagen.flow_from_dataframe(
    dataframe=gan_val_df,
    x_col='image',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH,
    shuffle=False,
    seed=SEED
)


ds_test = val_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='image',
    y_col='class',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=1,
    shuffle=False
)


Found 13175 validated image filenames belonging to 4 classes.
Found 2325 validated image filenames belonging to 4 classes.
Found 771 validated image filenames belonging to 4 classes.


In [ ]:
#need to split the train and validation data
#ds_train, ds_val,

IMG_SIZE = 150
BATCH = 32
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)



base_model = tf.keras.applications.ResNet152V2(
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False)

base_model.trainable = False

def get_pretrained():

    #Input shape = [width, height, color channels]
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    x = base_model(inputs)

    # Head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.1)(x)

    #Final Layer (Output)
    output = layers.Dense(4, activation='softmax')(x)

    model = keras.Model(inputs=[inputs], outputs=output)

    return model
keras.backend.clear_session()

model_pretrained = get_pretrained()
model_pretrained.compile(loss='categorical_crossentropy'
              , optimizer = keras.optimizers.Adam(learning_rate=5e-5), metrics=['binary_accuracy'])

model_pretrained.summary()
history = model_pretrained.fit(ds_train,
          batch_size = BATCH, epochs = 50,
          validation_data=ds_val,
          #callbacks=[early_stopping, plateau],
          steps_per_epoch=int((len(ds_train)/BATCH)),
          validation_steps=int((len(ds_val)/BATCH)))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet152v2 (Functional)        │ (None, 5, 5, 2048)     │    58,331,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,594,436 (223.52 MB)

 Trainable params: 262,788 (1.00 MB)

 Non-trainable params: 58,331,648 (222.52 MB)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 111s 8s/step - binary_accuracy: 0.6433 - loss: 2.0638 - val_binary_accuracy: 0.7031 - val_loss: 1.6481
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - binary_accuracy: 0.7376 - loss: 1.3679 - val_binary_accuracy: 0.8008 - val_loss: 1.1140
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - binary_accuracy: 0.7908 - loss: 1.0834 - val_binary_accuracy: 0.8477 - val_loss: 0.8516
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - binary_accuracy: 0.8232 - loss: 0.8889 - val_binary_accuracy: 0.8828 - val_loss: 0.7262
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 87s 7s/step - binary_accuracy: 0.8198 - loss: 0.8872 - val_binary_accuracy: 0.9062 - val_loss: 0.6505
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 88s 7s/step - binary_accuracy: 0.8532 - loss: 0.7429 - val_binary_accuracy: 0.9023 - val_loss: 0.6122
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 87s 7s/step - binary_accuracy: 0.8545 - loss: 0.7688 - val_binary_accuracy: 0.9297 - val_loss: 0.5336
Epoch 8/50
12/12 ━━

In [1]:
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['loss'])
sns.lineplot(x = history.epoch, y = history.history['val_loss'])
ax.set_title('Learning Curve (Loss)')
ax.set_ylabel('Loss')
ax.set_xlabel('Epoch')
ax.set_ylim(0, 0.5)
ax.legend(['train', 'val'], loc='best')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
sns.lineplot(x = history.epoch, y = history.history['binary_accuracy'])
sns.lineplot(x = history.epoch, y = history.history['val_binary_accuracy'])
ax.set_title('Learning Curve (Accuracy)')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.set_ylim(0.80, 1.0)
ax.legend(['train', 'val'], loc='best')
plt.show()

In [ ]:
score = model_pretrained.evaluate(ds_val, steps = len(val_df)/BATCH, verbose = 0)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

In [ ]:
score = model_pretrained.evaluate(ds_test, steps = len(df_test), verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])